In [11]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
%matplotlib inline

###  Can tissue associated microbiota profiles predict phenotype with greater accuracy than caecal content microbiota? 


In [3]:
# read in files and merge shannon.effective column from alpha-diverstiy table with otu table and classes from mapping file
def read_merge(otu, alpha_div, mapping, alpha_metric="Shannon.effective", mapping_classes="Phenotype"):
    otu_table = pd.read_table(otu, header=0, index_col=0).T
    mapping_file = pd.read_table(mapping, header=0, index_col=0)
    alpha_diversity = pd.read_table(alpha_div, header=0, index_col=0)
    merged = pd.concat([alpha_diversity[alpha_metric], otu_table, mapping_file[mapping_classes]], axis=1 )
    return merged

In [4]:
# merge caecal content files
cc_merged = read_merge("CC/OTUs_Table-norm-rel.tab", "CC/alpha-diversity.tab", "CC/mapping_file.tab")
# merge tissue associated files
tissue_merged = read_merge("tissue/OTUs_Table-norm-rel.tab", "tissue/alpha-diversity.tab", "tissue/mapping_file.tab")

In [5]:
X_cc = cc_merged.drop("Phenotype", axis=1)
y_cc = cc_merged.Phenotype
X_tissue = tissue_merged.drop("Phenotype", axis=1)
y_tissue = tissue_merged.Phenotype

### Classification 

**Random forest**

In [9]:
rf_cc = RandomForestClassifier(n_estimators=100,random_state=42, n_jobs=-1, oob_score=True)
rf_tissue = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, oob_score=True)

loocv = LeaveOneOut()
results_cc = cross_val_score(rf_cc, X_cc, y_cc, cv=loocv, n_jobs=-1)
results_tissue = cross_val_score(rf_tissue, X_tissue, y_tissue, cv=loocv, n_jobs=-1)
print("Accuracy: %.2f%% (%.2f%%)" % (results_cc.mean()*100.0, results_cc.std()*100.0))
print("Accuracy: %.2f%% (%.2f%%)" % (results_tissue.mean()*100.0, results_tissue.std()*100.0))

Accuracy: 90.74% (28.99%)
Accuracy: 98.39% (12.60%)


**Support vector machine**

In [7]:
svm_cc = SVC(random_state=42, gamma='auto', C=1)
svm_tissue = SVC(random_state=42, gamma='auto', C=1)

results_cc_svm = cross_val_score(svm_cc, X_cc, y_cc, cv=loocv, n_jobs=-1)
results_tissue_svm = cross_val_score(svm_tissue, X_tissue, y_tissue, cv=loocv, n_jobs=-1)
print("Accuracy: %.2f%% (%.2f%%)" % (results_cc_svm.mean()*100.0, results_cc_svm.std()*100.0))
print("Accuracy: %.2f%% (%.2f%%)" % (results_tissue_svm.mean()*100.0, results_tissue_svm.std()*100.0))

Accuracy: 81.48% (38.84%)
Accuracy: 91.94% (27.23%)


**K-nearest neighbour** 

In [8]:
knn_cc = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn_tissue = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

results_cc_knn = cross_val_score(knn_cc, X_cc, y_cc, cv=loocv, n_jobs=-1)
results_tissue_knn = cross_val_score(knn_tissue, X_tissue, y_tissue, cv=loocv, n_jobs=-1)
print("Accuracy: %.2f%% (%.2f%%)" % (results_cc_knn.mean()*100.0, results_cc_knn.std()*100.0))
print("Accuracy: %.2f%% (%.2f%%)" % (results_tissue_knn.mean()*100.0, results_tissue_knn.std()*100.0))

Accuracy: 79.63% (40.28%)
Accuracy: 91.94% (27.23%)


## With feature selection 

**Random Forest with feature selection**

In [23]:
X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(X_cc, y_cc, test_size=0.2)

clf_cc = Pipeline([
  ('feature_selection', SelectFromModel(RandomForestClassifier())),
  ('classification', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])
clf_cc.fit(X_train_cc, y_train_cc)

Pipeline(steps=[('feature_selection', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weig...imators=100, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False))])

In [24]:
results_clf = cross_val_score(clf_cc, X_train_cc, y_train_cc, cv=5, n_jobs=-1)
print("Accuracy: %.2f%% (%.2f%%)" % (results_clf.mean()*100.0, results_clf.std()*100.0))

Accuracy: 92.78% (5.92%)


In [25]:
X_train_tissue, X_test_tissue, y_train_tissue, y_test_tissue = train_test_split(X_tissue, y_tissue, test_size=0.2)

clf_tissue = Pipeline([
  ('feature_selection', SelectFromModel(RandomForestClassifier())),
  ('classification', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])
clf_cc.fit(X_train_tissue, y_train_tissue)

Pipeline(steps=[('feature_selection', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weig...imators=100, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False))])

In [26]:
results_clf2 = cross_val_score(clf_tissue, X_train_tissue, y_train_tissue, cv=5, n_jobs=-1)
print("Accuracy: %.2f%% (%.2f%%)" % (results_clf2.mean()*100.0, results_clf2.std()*100.0))

Accuracy: 93.78% (5.10%)
